
Introduction to TorchScript
===========================

In [1]:
import torch # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)

1.8.2+cu111


Basics of PyTorch Model Authoring
---------------------------------

Let’s start out be defining a simple ``Module``. A ``Module`` is the
basic unit of composition in PyTorch. It contains:

1. A constructor, which prepares the module for invocation
2. A set of ``Parameters`` and sub-\ ``Modules``. These are initialized
   by the constructor and can be used by the module during invocation.
3. A ``forward`` function. This is the code that is run when the module
   is invoked.

Let’s examine a small example:




In [2]:
x = torch.rand(1, 2)
h = torch.rand(1, 2)

In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(2, 2)

    def forward(self, x, h):
        lin = self.linear(x)
        new_h = torch.tanh(lin + h)
        return lin, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=2, out_features=2, bias=True)
)
(tensor([[-0.9252, -0.3446]], grad_fn=<AddmmBackward>), tensor([[-0.4981, -0.3001]], grad_fn=<TanhBackward>))


Let's add some more logic - decision gate )

In [4]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCellDG(torch.nn.Module):
    def __init__(self):
        super(MyCellDG, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(2, 2)

    def forward(self, x, h):
        lin = self.linear(x)
        new_h = torch.tanh(self.dg(lin) + h)
        return lin, new_h

my_cell_dg = MyCellDG()
print(my_cell_dg)
print(my_cell_dg(x, h))

MyCellDG(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=2, out_features=2, bias=True)
)
(tensor([[-0.1605, -0.3371]], grad_fn=<AddmmBackward>), tensor([[0.4922, 0.3558]], grad_fn=<TanhBackward>))


We’ve once again redefined our MyCell class, but here we’ve defined
``MyDecisionGate``. This module utilizes **control flow**. Control flow
consists of things like loops and ``if``-statements.

Many frameworks take the approach of computing symbolic derivatives
given a full program representation. However, in PyTorch, we use a
gradient tape. We record operations as they occur, and replay them
backwards in computing derivatives. In this way, the framework does not
have to explicitly define derivatives for all constructs in the
language.

.. figure:: https://github.com/pytorch/pytorch/raw/master/docs/source/_static/img/dynamic_graph.gif
   :alt: How autograd works

   How autograd works




Basics of TorchScript
---------------------

Now let’s take our running example and see how we can apply TorchScript.

In short, TorchScript provides tools to capture the definition of your
model, even in light of the flexible and dynamic nature of PyTorch.
Let’s begin by examining what we call **tracing**.

Tracing ``Modules``
~~~~~~~~~~~~~~~~~~~




In [5]:
traced_cell = torch.jit.trace(my_cell, (x, h))
# traced_cell_dg = torch.jit.trace(my_cell_dg, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[-0.9252, -0.3446]], grad_fn=<AddBackward0>),
 tensor([[-0.4981, -0.3001]], grad_fn=<TanhBackward>))

We’ve rewinded a bit and taken the second version of our ``MyCell``
class. As before, we’ve instantiated it, but this time, we’ve called
``torch.jit.trace``, passed in the ``Module``, and passed in *example
inputs* the network might see.

What exactly has this done? It has invoked the ``Module``, recorded the
operations that occured when the ``Module`` was run, and created an
instance of ``torch.jit.ScriptModule`` (of which ``TracedModule`` is an
instance)

TorchScript records its definitions in an Intermediate Representation
(or IR), commonly referred to in Deep learning as a *graph*. We can
examine the graph with the ``.graph`` property:




In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %input : Float(1, 2, strides=[2, 1], requires_grad=0, device=cpu),
      %h : Float(1, 2, strides=[2, 1], requires_grad=0, device=cpu)):
  %21 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %23 : Tensor = prim::CallMethod[name="forward"](%21, %input)
  %14 : int = prim::Constant[value=1]() # /tmp/ipykernel_7432/2142782686.py:8:0
  %15 : Float(1, 2, strides=[2, 1], requires_grad=1, device=cpu) = aten::add(%23, %h, %14) # /tmp/ipykernel_7432/2142782686.py:8:0
  %16 : Float(1, 2, strides=[2, 1], requires_grad=1, device=cpu) = aten::tanh(%15) # /tmp/ipykernel_7432/2142782686.py:8:0
  %17 : (Float(1, 2, strides=[2, 1], requires_grad=1, device=cpu), Float(1, 2, strides=[2, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%23, %16)
  return (%17)



However, this is a very low-level representation and most of the
information contained in the graph is not useful for end users. Instead,
we can use the ``.code`` property to give a Python-syntax interpretation
of the code:




In [7]:
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.linear).forward(input, )
  _1 = (_0, torch.tanh(torch.add(_0, h, alpha=1)))
  return _1



So **why** did we do all this? There are several reasons:

1. TorchScript code can be invoked in its own interpreter, which is
   basically a restricted Python interpreter. This interpreter does not
   acquire the Global Interpreter Lock, and so many requests can be
   processed on the same instance simultaneously.
2. This format allows us to save the whole model to disk and load it
   into another environment, such as in a server written in a language
   other than Python
3. TorchScript gives us a representation in which we can do compiler
   optimizations on the code to provide more efficient execution
4. TorchScript allows us to interface with many backend/device runtimes
   that require a broader view of the program than individual operators.

We can see that invoking ``traced_cell`` produces the same results as
the Python module:




In [8]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[-0.9252, -0.3446]], grad_fn=<AddmmBackward>), tensor([[-0.4981, -0.3001]], grad_fn=<TanhBackward>))
(tensor([[-0.9252, -0.3446]], grad_fn=<AddBackward0>), tensor([[-0.4981, -0.3001]], grad_fn=<TanhBackward>))


Using Scripting to Convert Modules
----------------------------------

There’s a reason we used version two of our module, and not the one with
the control-flow-laden submodule. Let’s examine that now:




In [9]:
class MyCellDGP(torch.nn.Module):
    def __init__(self, dg):
        super(MyCellDGP, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(2, 2)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h
      
my_cell = MyCellDGP(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.dg
  _1 = (self.linear).forward(input, )
  _2 = (_0).forward(_1, )
  _3 = torch.tanh(torch.add(_1, h, alpha=1))
  return (_3, _3)



/tmp/ipykernel_7432/3115164013.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


Looking at the ``.code`` output, we can see that the ``if-else`` branch
is nowhere to be found! Why? Tracing does exactly what we said it would:
run the code, record the operations *that happen* and construct a
ScriptModule that does exactly that. Unfortunately, things like control
flow are erased.

How can we faithfully represent this module in TorchScript? We provide a
**script compiler**, which does direct analysis of your Python source
code to transform it into TorchScript. Let’s convert ``MyDecisionGate``
using the script compiler:




In [10]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCellDGP(scripted_gate)
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  new_h = torch.tanh(torch.add(_0, h, alpha=1))
  return (new_h, new_h)



Hooray! We’ve now faithfully captured the behavior of our program in
TorchScript. Let’s now try running the program:




In [11]:
# New inputs
x, h = torch.rand(1, 2), torch.rand(1, 2)
traced_cell(x, h)

(tensor([[0.1346, 0.5547]], grad_fn=<TanhBackward>),
 tensor([[0.1346, 0.5547]], grad_fn=<TanhBackward>))

Mixing Scripting and Tracing
~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Some situations call for using tracing rather than scripting (e.g. a
module has many architectural decisions that are made based on constant
Python values that we would like to not appear in TorchScript). In this
case, scripting can be composed with tracing: ``torch.jit.script`` will
inline the code for a traced module, and tracing will inline the code
for a scripted module.

An example of the first case:




This way, scripting and tracing can be used when the situation calls for
each of them and used together.

Saving and Loading models
-------------------------

We provide APIs to save and load TorchScript modules to/from disk in an
archive format. This format includes code, parameters, attributes, and
debug information, meaning that the archive is a freestanding
representation of the model that can be loaded in an entirely separate
process. Let’s save and load our wrapped RNN module:




In [12]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCellDGP(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(1, 2), torch.zeros(1, 2)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([1, 2], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([1, 2], dtype=None, layout=None, device=None, pin_memory=None)
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    _0 = (self.cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [13]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 1, 2)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)



In [14]:
traced.save('wrapped_rnn.zip')

loaded = torch.jit.load('wrapped_rnn.zip')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCellDGP
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)



As you can see, serialization preserves the module hierarchy and the
code we’ve been examining throughout. The model can also be loaded, for
example, `into
C++ <https://pytorch.org/tutorials/advanced/cpp_export.html>`__ for
python-free execution.

Further Reading
~~~~~~~~~~~~~~~

We’ve completed our tutorial! For a more involved demonstration, check
out the NeurIPS demo for converting machine translation models using
TorchScript:
https://colab.research.google.com/drive/1HiICg6jRkBnr5hvK2-VnMi88Vi9pUzEJ


